금리/채권 지수

미국 10년 만기 국채 금리, 미국 연방기금 금리, 한국은행 기준 금리 지수가 있으며 60일치 데이터입니다.

1. 미국 10년 만기 국채 금리(tnx)

In [ ]:
!pip install --upgrade --force-reinstall numpy
!pip install --upgrade --force-reinstall yfinanc #아래 코드 에러나면 돌려주세요.

In [ ]:
import yfinance as yf
import pandas as pd

# 미국 10년물 국채 수익률 (^TNX) 60일치 데이터 다운로드
raw_tnx = yf.download('^TNX', period='60d', interval='1d')

# 날짜와 Close 컬럼만 선택, 인덱스에서 컬럼으로 변환
tnx = raw_tnx.reset_index()[['Date', 'Close']]

# 컬럼 이름 한글로 변경
tnx.columns = ['날짜', '금리']

# 결과 출력
tnx

C:\Users\mcdor\AppData\Local\Temp\ipykernel_10752\546648988.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  raw_tnx = yf.download('^TNX', period='60d', interval='1d')
[*********************100%***********************]  1 of 1 completed


,날짜,금리
0,2025-04-11,4.493
1,2025-04-14,4.364
2,2025-04-15,4.323
3,2025-04-16,4.279
4,2025-04-17,4.333
5,2025-04-21,4.405
6,2025-04-22,4.389
7,2025-04-23,4.387
8,2025-04-24,4.305
9,2025-04-25,4.266


2. 미국 연방기금 금리

fred api로 크롤링 작업했습니다

In [ ]:
!pip install fredapi #필요시 설치해주세요.


In [18]:
from fredapi import Fred
import pandas as pd
from datetime import datetime, timedelta

# FRED API 키 입력
fred = Fred(api_key='6091f9de7a869ea03c162ff55ae0763c')

# 오늘 날짜 기준 60일 전 날짜 계산
start_date = (datetime.today() - timedelta(days=60)).strftime('%Y-%m-%d')

# FEDFUNDS 시계열 데이터 60일치 불러오기 (실제 데이터 있는 날짜만)
fedfunds = fred.get_series('FEDFUNDS', observation_start=start_date).reset_index()
fedfunds.columns = ['날짜', '금리']

fedfunds

,날짜,금리
0,2025-05-01,4.33
1,2025-06-01,4.33


-> fred에서 7월 1일 데이터 가져올 수가 없음.

In [19]:
# 기존 fedfunds 데이터프레임 있다고 가정

# 7월 1일 데이터 생성 (날짜 타입 맞추기)
new_row = pd.DataFrame({'날짜': [pd.to_datetime('2025-07-01')], '금리': [4.33]})

# 기존 데이터와 합치기
fedfunds = pd.concat([fedfunds, new_row], ignore_index=True)

# 날짜 기준 정렬 (필요시)
fedfunds = fedfunds.sort_values('날짜').reset_index(drop=True)

fedfunds

,날짜,금리
0,2025-05-01,4.33
1,2025-06-01,4.33
2,2025-07-01,4.33


-> fred에서 7월 1일 데이터를 제공 아직 안 했다해서 그냥 제가 실제 금리 넣었어요.

3. 한국은행 기준 금리

In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# ECOS API 키
API_KEY = '8K9P1GUDEKM7AUB1KXJN'

# 기준금리 시계열 코드: 722Y001
# 통계항목 코드: 0101000 (한국은행 기준금리)
stat_code = '722Y001'
item_code = '0101000'
freq = 'M'  # 월별

# 최근 2개월 범위 지정 (YYYYMM 형식)
end_date = datetime.today().strftime('%Y%m')
start_date = (datetime.today() - timedelta(days=60)).strftime('%Y%m')

# API 요청 URL 구성
url = (
    f"http://ecos.bok.or.kr/api/StatisticSearch/{API_KEY}/json/kr/1/1000/"
    f"{stat_code}/{freq}/{start_date}/{end_date}/{item_code}"
)

# 요청 및 응답 처리
res = requests.get(url)
data = res.json()

# 에러 처리
if 'StatisticSearch' not in data:
    print("API 오류 또는 데이터 없음:", data)
else:
    rows = data['StatisticSearch']['row']
    df = pd.DataFrame(rows)[['TIME', 'DATA_VALUE']]
    df.columns = ['날짜', '기준금리']
    df['날짜'] = pd.to_datetime(df['날짜'], format='%Y%m')
    df['기준금리'] = df['기준금리'].astype(float)
    df = df.sort_values('날짜').reset_index(drop=True)

    print(df)


          날짜  기준금리
0 2025-05-01   2.5
1 2025-06-01   2.5
